In [91]:
# TODO: constraints; cost function? why nega score?; goal is to achieve comparable performance. 

In [1]:
# https://www.kaggle.com/pavelvod/pytorch-starter-solution
!pip install torch

    100% |████████████████████████████████| 71.1MB 657kB/s ta 0:00:011


In [120]:
import utils
import importlib

import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

In [414]:
# giftCostsNp (5000, 100): [i,j] means the gift cost when i-th family choses j-th day.
row, col = 5000, 100
giftCostsNp = np.zeros((row, col))
familySizesNp = np.zeros((row, 1))

familyDataNp = utils.readDataDf().values

for i in range(row):
    familySizesNp[i] = familyDataNp[i, -1]
    for j in range(col):
        giftCostsNp[i,j] = utils.getCurGift(assignDay=j+1, 
                                          familyPreferencesList=familyDataNp[i, 1:-1].tolist(), 
                                          familySize=familySizesNp[i])

In [ ]:
class Model(nn.Module):
    def __init__(self, giftCostsNp, familySizesNp):
        super().__init__()
        self.giftCostsTc = torch.from_numpy(giftCostsNp).type(torch.float32)
        self.familySizesTc = torch.from_numpy(familySizesNp).type(torch.float32)
        self.assignWeightsTc = torch.nn.Parameter(data=torch.Tensor(5000, 100), requires_grad=True)
        self.assignWeightsTc.data.uniform_(0,5) # weight initialization?
        
    def forward(self):
        assignProbTc = F.softmax(self.assignWeightsTc, dim=1)
#         np.savetxt("assignProbNp.txt", assignProbTc.data.numpy())
#         print(self.giftCostsTc.shape, assignProbTc.shape)
        giftCostTotalTc = (self.giftCostsTc * assignProbTc).sum()
        # giftCostsTc is 5000,100
        # assignProbTc is 5000, 100
        
        NdTc = assignProbTc.transpose(0,1) @ self.familySizesTc
        # NdTc = (assignProbTc.transpose(0,1) @ self.familySizesTc).sum(axis=1)
        # assignProbTc: 5000, 100
        # assignProbTc.transpose: 100, 5000 (0th row: for day-1, each family assigned how much prob. )
        # familySizesTc: 5000, 1 (0th col: each family's size. )
        # @ result: (100, 1) (0th col: each day's assigned prob weighted by size)
        # after sum: (1,1). WTF?!
        
        
#         print(F.softmax(NdTc).max(), F.softmax(NdTc).argmax())
        NdTcCat = torch.zeros((101,1), dtype=torch.float32)
        
        NdTcCat[:-1,0] = NdTc[:,0]
#         print(NdTcCat) 
        NdTcCat[-1,0] = NdTc[-1,0]
#         print(NdTcCat) 
        
#         print("exponent: ", 0.5+abs(NdTcCat[:-1]-NdTcCat[1:])/50.)
        cleanCostTotalTc = (torch.relu(NdTc-125.)/400. * NdTc**(0.5+abs(NdTcCat[:-1]-NdTcCat[1:])/50.)).sum()
        
        penaltyCostTotalTc = (torch.relu(125.-NdTc)**2 + torch.relu(NdTc-300.)**2).sum() * 1000000
        return giftCostTotalTc, cleanCostTotalTc, penaltyCostTotalTc, NdTc

In [422]:
# def cost_function():
#     return None
importlib.reload(utils)

<module 'utils' from '/Users/songxu/PycharmProjects/Data_Science/Kaggle/santa-workshop-tour-2019/utils.py'>

In [435]:
model1 = Model(giftCostsNp, familySizesNp)
best_score = 10e10
best_pos = None

In [436]:
optimizer = torch.optim.Adam(model1.parameters())
for epoch in range(1,150000):
    optimizer.zero_grad()
    x, y, z, NdTc = model1()
#     print(x.shape, y.shape)
    loss = x + y + z
    loss.backward()
    optimizer.step()
    
#     assignProbNp = F.softmax(model.assignWeightsTc, dim=1).data.numpy()    
#     np.savetxt("intermediate/assignProbNp.txt", assignProbNp)
    
#     break
    
    
    if epoch % 1000 == 0:
        
        # (5000, 100): for each family (keep dim 0), find the most-weighted day
        
#         print(model.assignWeightsTc[0,:])
#         print(model.assignWeightsTc[0,:].max(), model.assignWeightsTc[0,:].argmax())
        
#         print(assignments.max(), assignments.min())

        assignments = model1.assignWeightsTc.argmax(dim=1).numpy()+1
#         assignProbNp = F.softmax(model.assignWeightsTc, dim=1).data.numpy()    
#         np.savetxt("intermediate/assignProbNp.txt", assignProbNp)

#         NdTensorNp = NdTc.data.numpy()
#         np.savetxt("intermediate/NdTensorNp.txt", NdTensorNp)
#         with open(, "w") as fOut:
#             rowOut, colOut = assignProbNp.shape
#             for i in range(rowOut):
#                 fOut.write(assignProbNp[i, :])
#                 fOut.write("\n")
        print("gift, clean, penalty:", x.data.numpy(),y.data.numpy(),z.data.numpy())

        curScore = utils.getTotalScore(assignments, verbose=True)
        
        print(epoch, loss, curScore)
        print()
    
#         print("Here!", epoch, "loss:", loss, "score:", curScore)
    
#         print("NdTc[:10]:", NdTc[:10].data)
    
#         print(x, y, z)
#         print(curScore, z, loss)
#         break
#         pos = model.assignWeightsTc.argmax(1).numpy()
#         a, b, v = cost_function(pos+1)
#         score = a + b
#         if score < best_score:
#             best_score = score
#             best_pos = pos
#         x = np.round(x.item(), 3)
#         y = np.round(y.item(), 3)
#         print(f"{epoch}\t{x}\t{y}\t{score}\t{a}\t{b}\t{v}")

gift, clean, penalty: 7069079.5 28672.15 0.0
totGift, totClean: 1577899 193291407353.20126
1000 tensor(7097751.5000, grad_fn=<AddBackward0>) 193292985252.20126

gift, clean, penalty: 4161842.2 27331.234 0.0
totGift, totClean: 1086337 38862087.481408335
2000 tensor(4189173.5000, grad_fn=<AddBackward0>) 39948424.481408335

gift, clean, penalty: 2850495.0 13586.006 2.3283064
totGift, totClean: 864645 -226699024.39104867
3000 tensor(2864083.2500, grad_fn=<AddBackward0>) -225834379.39104867

gift, clean, penalty: 2121356.8 10264.638 0.0
totGift, totClean: 695745 35045020.90779152
4000 tensor(2131621.5000, grad_fn=<AddBackward0>) 35740765.90779152

gift, clean, penalty: 1640678.4 8471.3955 0.0
totGift, totClean: 557426 1372510.359038832
5000 tensor(1649149.7500, grad_fn=<AddBackward0>) 1929936.359038832

gift, clean, penalty: 1294422.0 7264.5493 0.0
totGift, totClean: 458571 20270034.4935333
6000 tensor(1301686.5000, grad_fn=<AddBackward0>) 20728605.4935333

gift, clean, penalty: 1035971.44 

gift, clean, penalty: 113715.31 5802.137 29.095041
totGift, totClean: 108782 -389136.29090449173
52000 tensor(119546.5469, grad_fn=<AddBackward0>) -280354.29090449173

gift, clean, penalty: 112866.68 5726.0825 14.320016
totGift, totClean: 108387 -392710.06457993167
53000 tensor(118607.0859, grad_fn=<AddBackward0>) -284323.06457993167

gift, clean, penalty: 112058.63 5814.0093 0.0
totGift, totClean: 106773 -392561.27488662204
54000 tensor(117872.6406, grad_fn=<AddBackward0>) -285788.27488662204

gift, clean, penalty: 111257.836 5821.703 0.0
totGift, totClean: 106229 -393440.9738434293
55000 tensor(117079.5391, grad_fn=<AddBackward0>) -287211.9738434293

gift, clean, penalty: 110478.43 6294.6284 0.0
totGift, totClean: 105758 -390466.3971816985
56000 tensor(116773.0547, grad_fn=<AddBackward0>) -284708.3971816985

gift, clean, penalty: 109720.055 5799.4956 0.0
totGift, totClean: 104838 -393974.0193994652
57000 tensor(115519.5469, grad_fn=<AddBackward0>) -289136.0193994652

gift, clean, pen

gift, clean, penalty: 92238.61 5708.4316 361.18326
totGift, totClean: 91630 -197780.6508616921
102000 tensor(98308.2188, grad_fn=<AddBackward0>) -106150.65086169209

gift, clean, penalty: 92027.39 5704.4585 0.013096724
totGift, totClean: 91680 -323597.6243644643
103000 tensor(97731.8672, grad_fn=<AddBackward0>) -231917.6243644643

gift, clean, penalty: 91835.34 6036.6797 0.15139812
totGift, totClean: 91186 -197682.45445398934
104000 tensor(97872.1719, grad_fn=<AddBackward0>) -106496.45445398934

gift, clean, penalty: 91651.08 5611.404 1.1736993
totGift, totClean: 91023 -197605.77641693145
105000 tensor(97263.6562, grad_fn=<AddBackward0>) -106582.77641693145

gift, clean, penalty: 91417.96 6182.1045 66.64195
totGift, totClean: 90750 -235876.47208881986
106000 tensor(97666.7031, grad_fn=<AddBackward0>) -145126.47208881986

gift, clean, penalty: 91228.78 5621.3647 0.0
totGift, totClean: 90248 -180887.21524026
107000 tensor(96850.1484, grad_fn=<AddBackward0>) -90639.21524026

gift, clean, 

KeyboardInterrupt: 

In [ ]:
loss

In [401]:
importlib.reload(utils)

<module 'utils' from '/Users/songxu/PycharmProjects/Data_Science/Kaggle/santa-workshop-tour-2019/utils.py'>

In [399]:
assignments

array([53, 27, 55, ..., 33, 93, 14])

In [402]:
utils.getTotalScore(assignments, verbose=True)

[99, 104, 242, 269, 300, 270, 249, 235, 258, 265, 301, 308, 301, 271, 256, 248, 275, 287, 282, 276, 242, 212, 220, 262, 283, 297, 305, 273, 255, 244, 244, 289, 271, 261, 227, 194, 176, 205, 224, 248, 235, 203, 174, 169, 203, 243, 269, 236, 216, 204, 179, 218, 246, 239, 211, 190, 151, 164, 203, 234, 230, 199, 165, 113, 122, 129, 242, 224, 186, 143, 127, 123, 110, 227, 203, 170, 130, 117, 132, 133, 219, 207, 177, 131, 124, 125, 131, 249, 229, 201, 157, 127, 130, 131, 314, 261, 215, 177, 132, 126]
totGift, totClean: 10588772 9452870.457375018


20041642.45737502

In [431]:
NdTc.data.numpy().tolist()

[[125.63609313964844],
 [225.6883087158203],
 [262.4656982421875],
 [287.7615966796875],
 [281.980712890625],
 [259.5467224121094],
 [251.5950469970703],
 [249.47315979003906],
 [267.9544677734375],
 [292.07012939453125],
 [299.1529541015625],
 [299.94976806640625],
 [279.8957214355469],
 [267.5335998535156],
 [261.9139404296875],
 [279.3463134765625],
 [299.634521484375],
 [298.6199035644531],
 [282.5728759765625],
 [256.8014831542969],
 [233.29623413085938],
 [238.39437866210938],
 [261.3100891113281],
 [286.94708251953125],
 [299.59796142578125],
 [299.19329833984375],
 [284.35491943359375],
 [269.676025390625],
 [254.2886505126953],
 [248.31134033203125],
 [275.40814208984375],
 [279.37115478515625],
 [259.62005615234375],
 [228.44033813476562],
 [201.4298095703125],
 [178.33648681640625],
 [202.716064453125],
 [236.244384765625],
 [259.9376220703125],
 [241.8492889404297],
 [211.04945373535156],
 [181.42010498046875],
 [183.7914581298828],
 [211.67327880859375],
 [247.530426025390

In [309]:
model_100000run = model